# Tableau Credential Management

#### This script can help manage credentials programatically. It includes 4 components: 
1. Add new users to multiple sites;
2. Change existing users' password or site role;
3. Get user info from multiple sites;
4. Get user info from all current sites (except some sites you really reall don't like)
5. Remove users from multiple sites.

## ***Add New Users

In [11]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def add_user(user_info):
    for site, username, fullname, siterole, password, email in user_info:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        #print 'Signin Succeed'
        t.add_user_by_username(username, site_role = siterole, update_if_exists=True)
        print '%s added to Site: %s'%(username, site)
    t.signout()
    print 'All Users Added'
    for site, username, fullname, siterole, password, email in user_info:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        users_in_server = t.query_users()
        for user in users_in_server:
            if user.get('name') == username:
                t.update_user_by_username(user.get('name'), password = password, full_name = fullname, email = email)
                print 'User: %s updated'%username
    t.signout()
    print 'All user updated'
    print 'Mission completed'

In [12]:
fileName = 'add_user.csv'
user_info = read_csv_file(fileName)
add_user(user_info)

test1 added to Site: TuneBIDemo
test2 added to Site: Uber
test3 added to Site: BetaTest
test4 added to Site: BetaTest2
All Users Added
User: test1 updated
User: test2 updated
User: test3 updated
User: test4 updated
All user updated
Mission completed


## *Update User Password

In [5]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginname = 'xiaojuan'
loginPass = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def update_passwd(userInfo):   
    for site, user, passwd in userInfo:
        t = TableauRestApi(server, loginname, loginPass, site)
        t.signin()
        t.update_user(user, password = passwd)
        print '%s Password Updated from %s'%(user, site)
    t.signout()
    print 'Mission Completed!!!'

In [6]:
fileName = 'ChangePassword.csv'
userInfo = read_csv_file(fileName)
update_passwd(userInfo)

yuri@tune.com Password Updated from Mixi
Sandor@tune.com Password Updated from Kabam
Sandor@tune.com Password Updated from PocketGems
Justin@tune.com Password Updated from Uber
Kristin-Leigh@tune.com Password Updated from Rhapsody
fatima@tune.com Password Updated from Xyrality
Miso@tune.com Password Updated from Mixi
Miso@tune.com Password Updated from 433
Miso@tune.com Password Updated from Netmarble
Daniella@tune.com Password Updated from AirRewardz
Daniella@tune.com Password Updated from Trippa
Daniella@tune.com Password Updated from CinetreatS
jv@tune.com Password Updated from AirRewardz
jv@tune.com Password Updated from Trippa
jv@tune.com Password Updated from CinetreatS
Neil@tune.com Password Updated from Ibotta
david@tune.com Password Updated from BetaTest
david@tune.com Password Updated from TuneBIDemo
colins@tune.com Password Updated from TuneBIDemo
nikki@tune.com Password Updated from TuneBIDemo
megank@tune.com Password Updated from TuneBIDemo
Demo Password Updated from TuneB

## *Change Site Role

In [27]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginname = 'xiaojuan'
loginPass = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def update_siterole(userInfo):   
    for site, user, siterole in userInfo:
        t = TableauRestApi(server, loginname, loginPass, site)
        t.signin()
        t.update_user(user, site_role = siterole)
        print '%s Site Role Updated from %s'%(user, site)
    t.signout()
    print 'Mission Completed!!!'

In [28]:
fileName = 'ChangeSiterole.csv'
userInfo = read_csv_file(fileName)
update_passwd(userInfo)

test1 Site Role Updated from TuneBIDemo
test2 Site Role Updated from Uber
test3 Site Role Updated from BetaTest
test4 Site Role Updated from BetaTest2
Mission Completed!!!


## *Get User Info and Generate CSV File

In [14]:
from tableau_rest_api.tableau_rest_api import *
import pandas as pd
import csv

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'

def read_csv_file(fileName):
    site_list = []
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        #header = read_file.next()
        for line in read_file:
            site_list.append(line)
    file.close()
    site_list = [i for site in site_list for i in site]
    return site_list

def user_info(site_list):
    username = []
    siterole = []
    lastLogin = []
    newSite = []
    valid_site_role = ['Guest', 'ServerAdministrator']
    for site in site_list:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        #print '%s Signin Succeed'%site
        for user in t.query_users():
            if user.get('siteRole') not in valid_site_role:
                username.append(user.get('name'))
                lastLogin.append(user.get('lastLogin'))
                siterole.append(user.get('siteRole'))
                newSite.append(site)
        print 'Get the User info from %s'%site
    #print username, siterole, lastLogin
    df_user = pd.DataFrame({'site':newSite, 'username':username, 'siteRole':siterole, 'lastLogin':lastLogin})
    df_user.to_csv('user.csv', sep = ',')
    t.signout()
    print 'CSV file generated'

In [15]:
site = read_csv_file('siteList.csv')
user_info(site)

Get the User info from TuneBIDemo
Get the User info from BetaTest
Get the User info from BetaTest2
Get the User info from Uber
Get the User info from 433
CSV file generated


## *Remove Users

In [11]:
from tableau_rest_api.tableau_rest_api import *
import csv

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'
user_info = []

def read_csv_file(fileName):
    with open(fileName, 'rU') as file:
        read_file = csv.reader(file)
        header = read_file.next()
        for line in read_file:
            user_info.append(line)
    file.close()
    return user_info

def remove_users(userInfo):
    for site, username in userInfo:
        t = TableauRestApi(server, loginName, loginPasswd, site)
        t.signin()
        users_in_server = t.query_users()
        for user in users_in_server:
            if user.get('name') != username:
                continue
            else:
                userId = user.get('id')
                t.remove_users_from_site_by_luid(userId)
                print '%s Removed from %s'%(username, site)
    t.signout()
    print 'Removed all users'
    print 'Mission Completed!!!'

In [12]:
userInfo = read_csv_file('remove_user.csv')
remove_users(userInfo)

test1 Removed from TuneBIDemo
test2 Removed from Uber
test3 Removed from BetaTest
test4 Removed from BetaTest2
Removed all users
Mission Completed!!!


## *Get user info of Almost all sites

In [42]:
from tableau_rest_api.tableau_rest_api import *
import pandas as pd

server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'

username = []
siterole = []
lastLogin = []
newSite = []
cur_date = []
invalid_site_role = ['Guest', 'ServerAdministrator', 'Unlicensed']
invalid_sites = ['Default', 'BetaTest', 'BetaTest2', 'WebDemo'] #Some sites I really don't care about


t = TableauRestApi(server, loginName, loginPasswd)
t.signin()
valid_site = [site for site in t.query_all_site_names() if site not in invalid_sites]
t.signout()

for site in valid_site:
    t = TableauRestApi(server, loginName, loginPasswd, site)
    t.signin()
    for user in t.query_users():
        if user.get('siteRole') not in invalid_site_role:
            username.append(user.get('name'))
            lastLogin.append(user.get('lastLogin'))
            siterole.append(user.get('siteRole'))
            newSite.append(site)
            cur_date.append()
    print 'Get the User info from %s'%site
#print username, siterole, lastLogin
df_user = pd.DataFrame({'site':newSite, 'username':username, 'siteRole':siterole, 'lastLogin':lastLogin})
df_user.to_csv('user.csv', sep = ',')
t.signout()
print 'CSV file generated'

Get the User info from Xyrality
Get the User info from Mixi
Get the User info from Ibotta
Get the User info from TuneBIDemo
Get the User info from AmericanWell
Get the User info from 433
Get the User info from Crowdstar
Get the User info from AirRewardz
Get the User info from Kabam
Get the User info from Trippa
Get the User info from Cinetreats
Get the User info from PocketGems
Get the User info from Uber
Get the User info from NetMarble
Get the User info from Uber-Fetch
Get the User info from Rhapsody
Get the User info from NextGames
Get the User info from Amazon
Get the User info from Match
CSV file generated


### ***Just Testing

In [29]:
from tableau_rest_api.tableau_rest_api import *
server = 'https://bi.tune.com'
loginName = 'xiaojuan'
loginPasswd = 'Has0ffers'
site = 'BetaTest'
t = TableauRestApi(server, loginName, loginPasswd)
t.signin()


In [36]:
invalid_sites = ['Default', 'BetaTest', 'BetaTest2', 'WebDemo']
sites = [site for site in t.query_all_site_names() if site not in invalid_sites]
sites

['Xyrality',
 'Mixi',
 'Ibotta',
 'TuneBIDemo',
 'AmericanWell',
 '433',
 'Crowdstar',
 'AirRewardz',
 'Kabam',
 'Trippa',
 'Cinetreats',
 'PocketGems',
 'Uber',
 'NetMarble',
 'Uber-Fetch',
 'Rhapsody',
 'NextGames',
 'Amazon',
 'Match']